## Covid19 Chest Xray Classifier Tutorial

## 0. common Import

In [2]:
import os #ls, cd, mkdir 같은 명령을 사용하게 하는거 
import random
import shutil # file관련된 사용 을 돕는 라이브러리
##---------------------------------표준라이브러리

import matplotlib.pyplot as plt
import numpy as np
import torch #파이토치 라이브러리
import torchvision #비전관련된 모델을 사용하도록 특화
torch.manual_seed(24)

#모델에서 특정 함수만 쓰고싶을 경우 from사용
from PIL import Image
from torchvision import transforms as T

ModuleNotFoundError: No module named 'matplotlib'

## 1. Prepare Training & Test Sets

In [ ]:
ROOT_DIR = os.path.abspath('.') #경로 저장, '.'의 .은 현재위치를 말함
DATA_DIR = os.path.join(ROOT_DIR,'datasets')
TEST_DIR = os.path.join(DATA_DIR, 'test')
os.makedirs(TEST_DIR, exist_ok=True)#exist_ok=True 만들고자하는 폴더의 이름이 있을경우 그대로 둔다

classes = ['normal','viral','covid']

for class_ in classes:
    start_dir = os.path.join(DATA_DIR, class_)
    target_dir = os.path.join(TEST_DIR, class_)
    if not os.path.exists(target_dir):
        os.makedirs(target_dir, exist_ok = True)
        images = os.listdir(start_dir)
        images = [image for image in images if image.lower().endswith('png')] #endswith 는 문자열이 있는지 확인해준다.
        sampled_images = random.sample(images, 30) #images 에서 30 개만 뽑아온다
        for image in sampled_images:
            start_path = os.path.join(start_dir, image) #파일명으로 해당 파일의 위치를 가져옴
            target_path = os.path.join(target_dir, image)
            shutil.move(start_path, target_path)

## 2.Define Custom Dataset

In [ ]:
train_dirs = {
    'normal': os.path.join(DATA_DIR,'normal'),
    'viral': os.path.join(DATA_DIR,'viral'),
    'covid': os.path.join(DATA_DIR,'covid')
}
transform = []
train_dataset = CovidCXRDataset(train_dirs,transform)

In [ ]:
# #getintem 사용하는 이유!
# class CustomContainer:
#     def __init__(self, container):
#         self.container = container 
#     def __len__(self):
#         return len(self.container) #주소만 출력됨 그래서 repr을 사용
#     def __repr__(self): 
#         return f'CustomContainer: {self.container}' #이대로만 하면 내부의 원소는 출력 안됨 그래서 getitem사용
#     def __getitem__(self,index):
#         return self.container[index]
# a = [1,2,3]    
# container = CustomContainer(a)
# print(container[1])

In [ ]:
class CovidCXRDataset(torch.utils.data.Dataset):
    def __init__(self, image_dirs, transform):
        self.images = {} 
        self.classes = ['normal','viral','covid']
        self.image_dirs = image_dirs # == train_dirs
        self.transform = transform
    #함수를 정의하는데, 이름은 get_images이고, 가져올 이미지들의 클래스를 인자로 받는다.
    #인자(argument; args) 파라미터(parameter; params)
        for class_ in self.classes: #1. class_ = 'normal'
            self.images[class_] = self.get_images(class_)
            #self.images['normal'] = self.get_images('normal')
            
    def get_images(self, class_): #'normal'이라는 값이 오면 self.image_dirs['normal'] ->경로가 설정되어있다.
        images = os.listdir(self.image_dirs[class_])
        #images = os.listdir(self.image_dirs['normal']) 여기서 image_dirs 이미지의 경로를 담았었다.
        #self.image_dirs == train_dirs
        images = [image for image in images
                 if image.lower().endswith('png')]#파이썬 안에서는 괄호안에서는 줄바꿈이 일어나도 에러가 나지않는다.
        print(f'Found {len(images)} {class_} examples')
        # Found 30 normal examples
        return images
#     self.images['normal'] -> normal이라는 폴더의 이미지를 가져온다

    def __len__(self):
        return sum([len(self.images[class_])
                    for class_ in self.classes]) #1. normal, 2.viral, 3.covid
    
    
    def __getitem__(self, index):
        # Pick class -> which?? 1.normal, 2.viral, 3.covid
        sampled_class = random.choice(self.classes)
        n_samples = len(self.images[sampled_class]) #Number of samples
        
        index = index % n_samples
        image_name = self.images[sampled_class] #sampled_class == 'normal' 이라면 그안에서 선정된 파일 명을 가져온다
        image_dir = self.image_dirs[sampled_class] #image_name의 파일이 있는 경로
        image_path = os.path.join(image_dir, image_name) #파일 이름과 경로를 붙여준다.
        image = Image.open(image_path).convert('RGB') #이미지 경로를 찾아가서 convert('RGB')사용해서 이미지객체로 만들어라
        return self.transform(image), self.classes.index(sampled_class) #index 가져오기
        #(Image_normal, 0)
        #(Image_covid, 2)
        #(Image_viral, 1)....으로 데이터셋 을 만든다

## 3. Create DataLoader

In [ ]:
train_transform = T.compose([
    T.resize(size=(224,224)), #resnet의 입력크기는 224x224 라서 그에 맞게 변경해준다.
    T.RandomHorizontalFlip(),#data augmentation 데이터에 약간의 변화를 줘서 과대적합을 막는다.
    T.ToTensor(), #이미지객체를 텐서로 전환 그래서 to tensor
    T.Normalize(mean = [0.485, 0.456, 0.406],
               std = [0.229, 0.224, 0.225]), # rgb 에 맞게 std를 red -> 0.229 green -> 0.224 ... 이런식으로 적용
])